## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime, date

## Download

In [6]:
url = "https://services5.arcgis.com/ROBnTHSNjoZ2Wm1P/arcgis/rest/services/COVID_19_Case_Rates_by_Zip_Code/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

## Parse

In [9]:
values = []

In [10]:
for f in data["features"]:
    row = f["attributes"].values()
    values.append(row)

KeyError: 'features'

In [96]:
cols = list(data["features"][0]["attributes"].keys())

Convert to dataframe

In [97]:
df = pd.DataFrame(values, columns=cols)

Get timestamp

In [98]:
date_url = "https://services5.arcgis.com/ROBnTHSNjoZ2Wm1P/ArcGIS/rest/services/COVID_19_Case_Rates_by_Zip_Code/FeatureServer/0/?f=json"

In [99]:
date_r = requests.get(date_url)

In [100]:
date_data = date_r.json()

In [101]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [102]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [103]:
update_date = pd.to_datetime(timestamp).date()

Add the timestamp to the dataframe

In [104]:
df["county_date"] = update_date

Rename and trim

In [105]:
clean_df = df[["Zip_Alpha", "Cases", "county_date"]].rename(
    columns={"Zip_Alpha": "zip", "Cases": "confirmed_cases"}
)

Match city names to zips

In [106]:
zips = [
    "94502",
    "94577",
    "94579",
    "94603",
    "94605",
    "94621",
    "94501",
    "94606",
    "94607",
    "94610",
    "94612",
    "94608",
    "94609",
    "94618",
    "94702",
    "94703",
    "94704",
    "94705",
    "94709",
    "94710",
    "94720",
    "94601",
    "94602",
    "94611",
    "94613",
    "94619",
    "94706",
    "94707",
    "94708",
    "94538",
    "94555",
    "94560",
    "94539",
    "94550",
    "94536",
    "94542",
    "94544",
    "94545",
    "94587",
    "94541",
    "94546",
    "94552",
    "94578",
    "94580",
    "94566",
    "94568",
    "94586",
    "94588",
    "94551",
    "95391",
    "94514",
    "94505",
    "95377",
]

In [107]:
cities = [
    "Alameda",
    "San Leandro",
    "San Leandro",
    "Oakland",
    "Oakland",
    "Oakland",
    "Alameda",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Emeryville",
    "Oakland",
    "Oakland",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Berkeley",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Oakland",
    "Albany",
    "Berkeley",
    "Berkeley",
    "Fremont",
    "Fremont",
    "Newark",
    "Fremont",
    "Livermore",
    "Fremont",
    "Hayward",
    "Hayward",
    "Hayward",
    "Union City",
    "Hayward",
    "Castro Valley",
    "Castro Valley",
    "San Leandro",
    "San Lorenzo",
    "Pleasanton",
    "Dublin",
    "Sunol",
    "Pleasanton",
    "Livermore",
    "Mountain House",
    "Byron",
    "Discovery Bay",
    "Tracy",
]

In [108]:
d = {"zip": zips, "city": cities}

In [109]:
matchup_df = pd.DataFrame(d)

In [110]:
merge_df = pd.merge(clean_df, matchup_df, how="left", on="zip")

In [111]:
merge_df["area"] = merge_df["zip"] + ": " + merge_df["city"]

Clean up

In [112]:
merge_df["county"] = "Alameda"

In [113]:
merge_df = merge_df[["county", "area", "county_date", "confirmed_cases"]]

Drop mysterious blank rows

In [114]:
filter_df = merge_df.dropna(subset=["area"])

Fill empty cases with 0

In [115]:
filter_df["confirmed_cases"] = filter_df.confirmed_cases.fillna(0)

Filter down to the latest date

In [116]:
export_df = filter_df[filter_df.county_date == filter_df.county_date.max()]

## Vet

In [117]:
try:
    assert not len(export_df) > 53
except AssertionError:
    raise AssertionError("Alameda scraper has additional rows than before")

In [118]:
try:
    assert not len(export_df) < 53
except AssertionError:
    raise AssertionError("Alameda County scraper is missing rows")

## Export

Mark the current date

In [119]:
tz = pytz.timezone("America/Los_Angeles")

In [120]:
today = datetime.now(tz).date()

In [121]:
slug = slugify("Alameda")

In [122]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [123]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [124]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [125]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [126]:
df.to_csv(os.path.join(data_dir / slug / "timeseries.csv"), index=False)